# Path (dapla)

In [ ]:
import os

import dapla as dp
import pandas as pd

os.chdir("../")
from fagfunksjoner import Path, PathSeries

In [ ]:
folder = Path('ssb-prod-kart-data-delt/kartdata_analyse/klargjorte-data/2023')
folder

In [ ]:
folder.ls()

In [ ]:
(
    folder.ls().files()
    .containing("ABAS_kommune")
    .loc[lambda x: x.mb > 100]
)

## Fungerer som tekst

In [ ]:
folder.startswith("ssb")

In [ ]:
"ssb" in folder

In [ ]:
dp.FileClient.get_gcs_file_system().exists(folder)

In [ ]:
file = folder / "ABAS_kommune_utenhav_p2023_v1.parquet"

dp.read_pandas(file, columns=["KOMMUNENR"])

## Med metoder og attributter ala pathlib.Path

In [ ]:
folder.exists()

In [ ]:
folder.is_dir()

In [ ]:
file = folder / "ABAS_kommune_utenhav_p2023_v1.parquet"
file

In [ ]:
file.parent

## Og noen pandas attributter

Uten å lese filen

In [ ]:
file.columns

In [ ]:
file.dtypes

In [ ]:
file.shape

## Versjonering

In [ ]:
file.version_number

In [ ]:
file.versions()

In [ ]:
file.latest_version()

In [ ]:
file.highest_numbered_version()

In [ ]:
# highest_numbered_version + 1
file.new_version()

In [ ]:
# alltid False
file.new_version().exists()

In [ ]:
# finner/fjerner versjonsnummer med regex
file._version_pattern

## Periode

In [ ]:
file.periods

In [ ]:
file.with_periods("2023-01")

In [ ]:
file.with_periods("2023-01-01")

In [ ]:
file.with_periods("2023-01-01").periods

In [ ]:
file.with_periods("2023", "2024")

In [ ]:
file.with_periods("2023", "2024").periods

In [ ]:
# finn riktig fil utfra stammen
year = 2023
stem = "ABAS_kommune_utenhav"

file = (
    (folder / stem)
    .with_periods(year)
    .highest_numbered_version()
)
file

In [ ]:
# feilmelding hvis filen ikke finnes
year = 2023
stem = "ABAS_hav"

file = (
    (folder / stem)
    .with_periods(year)
    .highest_numbered_version()
)
file

In [ ]:
# feilmelding hvis mer enn én fil matcher 'name'

year = 2023
stem = "ABAS_kommune"

file = (
    (folder / stem)
    .highest_numbered_version()
)
file

## Mer om ls

Stiene kopieres (som ctrl + c) når man klikker på stien.

In [ ]:
files_in_dir = file.parent.ls()
files_in_dir

In [ ]:
# subclass av pandas.Series
type(files_in_dir)

In [ ]:
files_in_dir.loc[lambda x: x.gb > 10].keep_latest_versions()

In [ ]:
# stiene er fortsatt Path
type(files_in_dir.iloc[0])

In [ ]:
# velg ut filene
folder.ls().files().keep_highest_numbered_versions()

In [ ]:
folder.parent.dirs()

In [ ]:
# samme som .loc med x.str.contains
folder.ls().containing("kommune")

In [ ]:
# og samme som
folder.glob("*kommune*")

In [ ]:
folder.ls().within_days(100)

In [ ]:
file.parent.parent.ls(recursive=True).files()

In [ ]:
folders: PathSeries = file.parent.parent.ls()
folders.ls_dirs()

In [ ]:
pd.concat(folders.ls_dirs())

## Write to testpath

In [ ]:
testpath = Path(
    'ssb-prod-dapla-felles-data-delt/path_test/test_df_p2023.parquet'
)

# delete files first
for version in testpath.versions():
    version.rm_file()

testpath = testpath.new_version()

testpath.exists()

In [ ]:
testpath

In [ ]:
df = pd.DataFrame({"x": [1,2,3], "y": [*"abc"]})

dp.write_pandas(df, testpath)

testpath.exists()

In [ ]:
testpath.latest_version()

In [ ]:
# highest_numbered_version + 1
testpath.new_version(timeout=None)

In [ ]:
# default med 30 minutter timeout for å ikke lagre i loop e.l.
for _ in range(5):
    dp.write_pandas(df, testpath.new_version())

In [ ]:
for _ in range(5):
    dp.write_pandas(df, testpath.new_version(timeout=None))

In [ ]:
dp.write_pandas(df, testpath.with_version(101))

In [ ]:
testpath.versions()